In [1]:
import numpy as np
import pandas as pd

In [3]:
# Data preparation

df_1 = pd.read_csv('res1.csv')
df_2 = pd.read_csv('res2.csv')
df_3 = pd.read_csv('res3.csv')

# df_1
df_1 = df_1.loc[pd.isna(df_1['price promo']) == False] # remove products that are not on sale
df_1['EAN'] = df_1['EAN'].apply(lambda x: x.replace(' ','')) # remove spaces from EAN
df_1 = df_1.loc[df_1['Name'] != 'Name'] # one weird record
df_1 = df_1.astype({'price':'float', 'price promo':'float'})

# df_2
df_2["price"] = np.where(pd.isna(df_2['price']) == True, df_2["price promo"], df_2["price"]) # price shift to the left. On the website, the promo price is as regural 
df_2["price promo"] = np.where(df_2["price promo"] == df_2["price"], '', df_2['price promo'])
df_2 = df_2.loc[df_2['price promo'] != ''] # remove products that are not on sale
df_2['price'] = df_2['price'].apply(lambda x: x[0] + x[2:] if len(x) > 6 else x) # if price bigger than 1000.00 it is as 1.000.00 so i remove second character
df_2 = df_2.loc[df_2['Name'] != 'Name']
df_2 = df_2.astype({'price promo':'float','price':'float','EAN':'str'})

# df_3
df_3["price"] = np.where(pd.isna(df_3['price']) == True, df_3["price promo"], df_3["price"]) # price shift to the left. On the website, the promo price is as regural 
df_3["price promo"] = np.where(df_3["price promo"] == df_3["price"], np.nan, df_3['price promo'])
df_3 = df_3.loc[(pd.isna(df_3['price promo']) == False) & (pd.isna(df_3['price']) == False)] # remove products that are not on sale; no price = product unavailable
df_3 = df_3.loc[df_3['Name'] != 'Name']
for i in ['price','price promo']:
    df_3[i] = df_3[i].apply(lambda x: x.replace(',','.').replace('\xa0','') if type(x) == str else x)
df_3 = df_3.astype({'price promo':'float','price':'float','EAN':'str'})

In [3]:
df_1 = df_1.rename(columns={"Name": "Name 1","price": "price 1", "price promo": "promo 1","url": "url 1"})
df_2 = df_2.rename(columns={"Name": "Name 2","price": "price 2", "price promo": "promo 2","url": "url 2"})
df_3 = df_3.rename(columns={"Name": "Name 3","price": "price 3", "price promo": "promo 3","url": "url 3"})

In [4]:
# Merging on EAN (It's possible that the same product is in two stores)
df = df_1.merge(df_2, how = 'outer', on = 'EAN').merge(df_3, how = 'outer', on = 'EAN').reset_index(drop = True)
df.head()

,Name 1,EAN,price 1,promo 1,url 1,Name 2,price 2,promo 2,url 2,Name 3,price 3,promo 3,url 3
0,DAVIDOFF The Game,3607349326135,219.99,153.99,https://www.rossmann.pl/Produkt/Wody-toaletowe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DAVIDOFF The Game,3607349326135,219.99,153.99,https://www.rossmann.pl/Produkt/Wody-toaletowe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HUGO BOSS Reversed,3614225296536,269.99,201.99,https://www.rossmann.pl/Produkt/Wody-toaletowe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HUGO BOSS Reversed,3614225296536,269.99,201.99,https://www.rossmann.pl/Produkt/Wody-toaletowe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HUGO BOSS Selection,3616301623298,249.99,184.99,https://www.rossmann.pl/Produkt/Wody-toaletowe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# lowest price
df['Price prom'] = df[['promo 1', 'promo 2', 'promo 3']].min(axis=1)

In [6]:
# if the lowest price is from a given store, then I select other information from that store as well
conditions = [
    (df['Price prom'] == df['promo 1']),
    (df['Price prom'] == df['promo 2']),
    (df['Price prom'] == df['promo 3']),
    ]

values_name = [df['Name 1'], df['Name 2'], df['Name 3']]
values_price = [df['price 1'], df['price 2'], df['price 3']]
values_url = [df['url 1'], df['url 2'], df['url 3']]

df['Product'] = np.select(conditions, values_name)
df['Price'] = np.select(conditions, values_price)
df['url'] = np.select(conditions, values_url)

In [7]:
df['Discount'] = df['Price'] - df['Price prom']

In [8]:
# sorting by biggest discounts
df = df[["Product", "Price", "Price prom", "Discount", "url"]]
df = df.sort_values(by='Discount', ascending = False).reset_index(drop=True)
df.head()

,Product,Price,Price prom,Discount,url
0,Roja Parfums Great Britain perfumy unisex,9870.30,5327.6,4542.70,https://www.notino.pl/roja-parfums/roja-parfum...
1,Roja Parfums Roja perfumy unisex,14829.10,12338.8,2490.30,https://www.notino.pl/roja-parfums/roja-perfum...
2,Roja Parfums Parfum de la Nuit 1 perfumy unisex,5783.35,3550.4,2232.95,https://www.notino.pl/roja-parfums/parfum-de-l...
3,Clive Christian No. 1 woda perfumowana dla męż...,3753.65,1852.0,1901.65,https://www.notino.pl/clive-christian/no-1-wod...
4,Clive Christian No. 1 woda perfumowana dla męż...,3753.65,1852.0,1901.65,https://www.notino.pl/clive-christian/no-1-wod...


In [9]:
# Create excel
with pd.ExcelWriter('Discounts.xlsx') as writer:
    df.to_excel(writer, sheet_name='Discounts', index = False, freeze_panes = [1,0])

In [10]:
# Sheet formatinf 
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font, Alignment

wb = load_workbook('Discounts.xlsx')
ws = wb['Discounts']

# header formatting
for cell in ws[1]:
    cell.fill = PatternFill(start_color='8FBC8F', fill_type="solid") 
    cell.alignment = Alignment(horizontal='left')
    cell.font = Font(color='FFFFFF', bold = True)
    
# width of the first column
ws.column_dimensions['A'].width = 30

# convert text url to hyperlink
for i in range(0, len(df)):
    ws.cell(row=i+2, column=5).hyperlink = str(df.iloc[i,4])
    ws.cell(row=i+2, column=5).value = "Link"
    ws.cell(row=i+2, column=5).style = "Hyperlink"

# coloring every other row
for i in range(3, len(df)+2, 2):  
    for cell in ws[i]:
        cell.fill = PatternFill(start_color='eafaea', fill_type="solid")

wb.save('Discounts.xlsx')
wb.close()

In [11]:
# e-mail with information about promotions
import os
import win32com.client as win32
from pretty_html_table import build_table, pretty_html_table

outlook = win32.Dispatch('outlook.application')

emails = ['...@email.com']

for i in emails:
    
    # html table
    df_html = build_table(df, 'green_light',font_size = '11px'
                                  ,font_family = 'Nano Sans, sans-serif'
                                  ,text_align = 'left'
                                  ,width_dict = ['220px','auto','70px','auto', '600px']
                                  ,padding = '5px')

    # send email
    Msg = outlook.CreateItem(0)
    Msg.To = i
    Msg.Subject = "Discounts"
    # it's necessary to set a precise path
    Msg.Attachments.Add(Source = os.getcwd() + '\\Discounts.xlsx')
    body = ( "<h3>Discounts</h3><br><br>" + df_html)
    Msg.HTMLBody = body
    Msg.Send()
    #Msg.display(True)